# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

In [2]:
import sqlite3
import csv
from datetime import datetime
DATA_DIR = "06_database_data/"
DATA_DIR_OUTPUT = "data_output/"

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [2]:
connection= sqlite3.connect(DATA_DIR+ 'Chinook_Sqlite.sqlite') #создаем подключение к базе данных 
cursor = connection.cursor() #  Создаем объект-курсор 
query = """SELECT FirstName, LastName FROM Customer WHERE Country == 'Canada'""" # создаем запрос
#print(query)
cursor.execute(query) #выполнение запроса
cursor.fetchall() #возвращает список всех (или всех оставшихся) записей из результата запроса.

[('François', 'Tremblay'),
 ('Mark', 'Philips'),
 ('Jennifer', 'Peterson'),
 ('Robert', 'Brown'),
 ('Edward', 'Francis'),
 ('Martha', 'Silk'),
 ('Aaron', 'Mitchell'),
 ('Ellie', 'Sullivan')]

2. Найти и вывести на экран названия всех альбомов группы Accept

In [3]:
query_Accept = """SELECT Title FROM Album 
WHERE ArtistId = (SELECT ArtistId FROM Artist Where Name = 'Accept')"""
cursor.execute(query_Accept)
cursor.fetchall()

[('Balls to the Wall',), ('Restless and Wild',)]

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [4]:
connect_2 = sqlite3.connect(DATA_DIR + 'pi194.sqlite')
cursor = connect_2.cursor()

query_new_table = """CREATE TABLE IF NOT EXISTS Student(
                     id integer NOT NULL PRIMARY KEY,
                     name VARCHAR(255) NOT NULL)"""
cursor.execute(query_new_table)

In [5]:
query_delete = """DELETE FROM Student"""
query_new_insert = """INSERT INTO Student (name) VALUES ('Michael Kuznetsov')"""
query_new_insert_2 = """INSERT INTO Student (name) VALUES ('Yevgen Bard')"""
print(query_delete)
print(query_new_insert)
print(query_new_insert_2)
cursor.execute(query_delete)
cursor.execute(query_new_insert)
cursor.execute(query_new_insert_2)
connect_2.commit()

query_to_watch = """SELECT * FROM Student"""
cursor.execute(query_to_watch)
#a = cursor.fetchone() # одна запись из запроса
a = cursor.fetchall() # все записи из запроса
connect_2.close() #Закрываем базу данных
a

DELETE FROM Student
INSERT INTO Student (name) VALUES ('Michael Kuznetsov')
INSERT INTO Student (name) VALUES ('Yevgen Bard')


[(1, 'Michael Kuznetsov'), (2, 'Yevgen Bard')]

In [6]:
#проверяем записали или нет данные 
connect_2 = sqlite3.connect(DATA_DIR + 'pi194.sqlite')
cursor = connect_2.cursor()


In [7]:
cursor.execute("""SELECT * FROM Student""")
cursor.fetchall()


[(1, 'Michael Kuznetsov'), (2, 'Yevgen Bard')]

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [3]:
connect = sqlite3.connect(DATA_DIR_OUTPUT + "recipes.db")
cursor = connect.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [9]:
create_recipe_db = """
CREATE TABLE IF NOT EXISTS Recipe (
    id integer not null,
    name varchar,
    minutes real,
    submitted integer,
    description text,
    n_ingredients integer,
    constraint 'recipe_pk' primary key (id)
)
"""
cursor.execute(create_recipe_db)
connect.commit()
# my_cursor.execute(droptable("Recipe"))

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [10]:
# droptable("Review")
create_review_db = """
CREATE TABLE IF NOT EXISTS Review (
    id integer not null,
    user_id integer,
    recipe_id integer,
    date integer,
    rating real,
    review text,
    constraint 'review_pk' primary key (id),
    constraint 'recipe_fk' foreign key (recipe_id) references Recipe(id)
)
"""
cursor.execute(create_review_db)
connect.commit()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [35]:
def check_date(string):
    return int(datetime.strptime(string, "%Y-%m-%d").timestamp()) # приводим дату к инту
    
with open(DATA_DIR + 'reviews_sample.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader) #пропуск 1 столбца
    #массив функций, проверяющих тип данных
    cast_types = [
        int,
        int, 
        int,
        check_date,
        float,
        str
    ]
    d = [[cast_types[i](row[i]) for i in range(len(row))] for row in reader]
d

[[370476,
  21752,
  57993,
  1051736400,
  5.0,
  "Last week whole sides of frozen salmon fillet was on sale in my local supermarket, so I bought tons (okay, only 3, but total weight was over 10 pounds).  This recipe is perfect for salmon fillet, even though it calls for salmon steaks.  I cut up the salmon into individual portions and followed the instructions exactly.  I'm on one of those food combining diets, so I left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  Super yummy, and leftovers for lunch today (lucky me)!"],
 [624300,
  431813,
  142201,
  1189890000,
  5.0,
  "So simple and so tasty!  I used a yellow capsicum in place of the green because that's what I had on hand.  This came together so fast.  Perfect meal if you don't have a lot of time.  Easy, healthy and tasty.  Thanks Stardustannie!  Made for PAC Fall 2007."],
 [187037,
  400708,
  252013,
  1199912400,
  4.0,
  'Very nice b

In [12]:
query_insert_reviews_sample = """INSERT INTO Review(id, user_id, recipe_id, date, rating, review) VALUES(?, ?, ?, ?, ?, ?)""" 
cursor.executemany(query_insert_reviews_sample,d) #выполняет SQL-зaпpoc несколько раз, при этом подставляя значения из последовательности.
connect.commit()


In [36]:
#Покажем что записали данные

query = """SELECT * FROM Review"""
cursor.execute(query) 
cursor.fetchall() #все данные из запроса

[(5,
  52282,
  120345,
  1116622800,
  4.0,
  "very very sweet. after i waited the 2 days i bought 2 more pints of raspberries and added them to the mix. i'm going to add some as a cake filling today and will take a photo."),
 (6,
  124416,
  120345,
  1312578000,
  0.0,
  'Just an observation, so I will not rate.  I followed this procedure with strawberries instead of raspberries.  Perhaps this is the reason it did not work well.  Sorry to report that the strawberries I did in August were moldy in October.  They were stored in my downstairs fridge, which is very cold and infrequently opened.  Delicious and fresh-tasting prior to that, though.  So, keep a sharp eye on them.  Personally I would not keep them longer than a month.  This recipe also appears as #120345 posted in July 2009, which is when I tried it.  I also own the Edna Lewis cookbook in which this appears.'),
 (7,
  2000192946,
  120345,
  1431205200,
  2.0,
  'This recipe was OVERLY too sweet.  I would start out with 1/3 

In [14]:
#RECIPES

def check_date(string):
    return int(datetime.strptime(string, "%Y-%m-%d").timestamp()) # приводим дату к инту

with open(DATA_DIR + "recipes_sample_with_tags_ingredients.csv", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",")
    next(reader)  #пропуск 1 столбца
    
    cast = [
        int, #номер строки
        str, #name
        int, #id
        float, #minutes
        int, #cont_id
        check_date, #date
        float, #n_steps
        str, #description
        int, #n_tags
        str, #tags
        int, #n_ingredients
        str, #ingredients
        
    ]
    a = [[cast[i](row[i]) for i in range(len(row))] for row in reader]
a
# Нам нужны только 
#id integer not null, [2]
#    name varchar, [1]
#    minutes real, [3]
#    submitted integer, [5]
#    description text, [7]
#    n_ingredients integer, [10]

[[0,
  'george s at the cove  black bean soup',
  44123,
  90.0,
  35193,
  1035493200,
  11.0,
  "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!",
  25,
  'weeknight;time-to-make;course;main-ingredient;cuisine;preparation;occasion;north-american;soups-stews;beans;poultry;american;chicken;stove-top;dietary;gluten-free;comfort-food;californian;black-beans;free-of-something;meat;taste-mood;equipment;grilling;4-hours-or-less',
  18,
  'unsalted butter*carrot*onion*celery*broccoli stem*dried thyme*dried oregano*dried sweet basil leaves*dry white wine*chicken stock*worcestershire sauce*tabasco sauce*smoked chicken*black beans*broccoli floret*heavy cream*salt & fresh ground pepper*cornstarch'],
 [1,
  'healthy fo

In [15]:
for row in a:
    row.pop(0)
    row.pop(3)
    row.pop(4)
    row.pop(5)
    row.pop(5)
    row.pop(6)
a


[['george s at the cove  black bean soup',
  44123,
  90.0,
  1035493200,
  "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!",
  18],
 ['healthy for them  yogurt popsicles',
  67664,
  10.0,
  1059166800,
  'my children and their friends ask for my homemade popsicles morning, noon and night. i never turn them down; who am i to tell them that they are good for them! for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.',
  3],
 ['i can t believe it s spinach',
  38798,
  30.0,
  1030568400,
  'these were so go, it surprised even me.',
  8],
 ['italian  gut busters',
  35173,
  45.0,
  1027717200,
  'my sister-in-law made these for us at a family get together. they are

In [16]:
c=0
for row in a:
    c = row[0]
    row[0] = row[1]
    row[1] = c
a

[[44123,
  'george s at the cove  black bean soup',
  90.0,
  1035493200,
  "an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!",
  18],
 [67664,
  'healthy for them  yogurt popsicles',
  10.0,
  1059166800,
  'my children and their friends ask for my homemade popsicles morning, noon and night. i never turn them down; who am i to tell them that they are good for them! for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.',
  3],
 [38798,
  'i can t believe it s spinach',
  30.0,
  1030568400,
  'these were so go, it surprised even me.',
  8],
 [35173,
  'italian  gut busters',
  45.0,
  1027717200,
  'my sister-in-law made these for us at a family get together. they are

In [17]:
query = """INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients) VALUES (?, ?, ?, ?, ?, ?)"""
cursor.executemany(query, a)
connect.commit()


In [4]:
# ПОКАЖЕМ ЗАПИСИ В RECIPE

cursor.execute("""SELECT * FROM Recipe""")
cursor.fetchall()

[(48, 'boston cream pie', 135.0, 935442000, '', 15),
 (55, 'betty crocker s southwestern guacamole dip', 125.0, 937342800, '', 5),
 (66,
  'black coffee barbecue sauce',
  30.0,
  936910800,
  "it's great to know folks like this sauce so much! i wish i could take credit for it but it was adopted after the original member left. thank you, all, just the same.\ni have made the recipe with regular brewed coffee but highly recommend the espresso. in a pinch use strong instant if you have nothing else. the acid of the coffee balances the sweetness of the other ingredients.",
  11),
 (91,
  'brown rice and vegetable pilaf',
  150.0,
  936565200,
  'this is good with almost anything... robb',
  13),
 (94, 'blueberry buttertarts', 40.0, 937083600, '', 8),
 (128, 'caramel apple milkshakes', 25.0, 937083600, '', 5),
 (148,
  'bugwiches',
  40.0,
  934318800,
  "a little bit of fun for the kids this summer. happy hunting! deborah   bugwiches:  whatever filling you choose to put between the top and

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [5]:
query = """SELECT * FROM Recipe WHERE n_ingredients == 10""" #или """SELECT * FROM Recipe WHERE n_ingredients == 10 LIMIT 5"""
cursor.execute(query)
cursor.fetchmany(5) #первые 5 

[(176, 'cola cake', 55.0, 935269200, '', 10),
 (246, 'lee s hot crab dip', 45.0, 936133200, "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150.0,
  938984400,
  'brazilian bean soup',
  10),
 (373,
  'tiramisu   balducci s in new york city',
  95.0,
  934837200,
  'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.',
  10),
 (544, 'greek mushroom salad', 38.0, 934232400, '', 10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени


In [6]:
#cursor.execute()
query = """SELECT NAME FROM Recipe WHERE minutes = (SELECT MAX(minutes) FROM Recipe)"""
print(query)
cursor.execute(query)
cursor.fetchone()

SELECT NAME FROM Recipe WHERE minutes = (SELECT MAX(minutes) FROM Recipe)


('strawberry liqueur',)

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [8]:
your_id = [int(input())]
query = """SELECT * FROM Recipe WHERE id = ?"""
print(query)
cursor.execute(query, your_id)
a = cursor.fetchall()
if (len(a) == 0): #или a == [] 
    print("Нет рецепта с выбранным id")
else:
    print("Рецепт нашелся!")
    print(a)

373
SELECT * FROM Recipe WHERE id = ?
Рецепт нашелся!
[(373, 'tiramisu   balducci s in new york city', 95.0, 934837200, 'this is an adopted recipe. i shall be trying it out and then post any additional information if needs be. if you are going to give any or no stars, please at least give the reason why so that you can help me improve the recipe. thank you.', 10)]


8. Найдите кол-во отзывов с рейтингом 5.

In [9]:
query = """SELECT COUNT(*) FROM Review WHERE rating = 5.0"""
cursor.execute(query)
cursor.fetchall()

[(91361,)]

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [13]:
query = """SELECT id FROM Review WHERE rating >= 4.0 AND review IS NULL"""
cursor.execute(query)
cursor.fetchall()

[]

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [16]:
n2009 = int(datetime.strptime('2009', "%Y").timestamp())
n2009

1230757200

In [18]:
n2010 = int(datetime.strptime('2010-01-01', "%Y-%m-%d").timestamp())
n2010

1262293200

In [19]:
n2010_v2 = int(datetime.strptime('2010-12-31', "%Y-%m-%d").timestamp())
n2010_v2

1293742800

In [20]:
n2011 = int(datetime.strptime('2011', "%Y").timestamp())
n2011

1293829200

In [22]:
#Правильный способ
query = """SELECT COUNT(*) FROM Recipe
            WHERE minutes >= 15.0 AND submitted BETWEEN ? AND ?"""
cursor.execute(query, [n2010,n2010_v2])
cursor.fetchall()

[(1319,)]

In [23]:
#2 способ
query = """SELECT Count(*) FROM Recipe
            WHERE minutes >= 15.0 AND strftime('%Y', datetime(submitted, 'unixepoch')) == '2010'"""
cursor.execute(query)
cursor.fetchall()

[(1318,)]

In [24]:
#3 способ
query = """SELECT COUNT(*) FROM Recipe
            WHERE minutes >= 15.0 AND submitted BETWEEN 1262293200 AND 1293742800"""
cursor.execute(query)
cursor.fetchall()

[(1319,)]

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [25]:
query = """SELECT Recipe.id, Recipe.name, user_id, date, rating From Review
            Inner JOIN Recipe
            ON Recipe.id = Review.id
            WHERE n_ingredients >= 3
            AND review IS NOT NULL 
            AND rating IS NOT NULL 
            AND date IS NOT NULL 
            ORDER BY (Recipe.id) ASC"""
cursor.execute(query)
cursor.fetchall()

[(246, 'lee s hot crab dip', 487088, 1359838800, 5.0),
 (318, 'french pecan pie ii', 68460, 1071867600, 4.0),
 (321, 'drunken chicken ii', 149342, 1088974800, 3.0),
 (373, 'tiramisu   balducci s in new york city', 650039, 1201381200, 4.0),
 (387, 'chocolate coffee ice cream soda', 41578, 1108155600, 5.0),
 (445, 'remarkable rhubarb bites', 29655, 1275426000, 5.0),
 (659, 'drunken chicken marinade', 39334, 1202590800, 5.0),
 (1075, 'beef crumble', 362983, 1182978000, 5.0),
 (2560, 'confetti coleslaw', 1858347, 1390770000, 0.0),
 (2633, 'beef  pepper   mushroom kabobs', 341170, 1254862800, 3.0),
 (2704, 'banana sultana muffins', 587675, 1280869200, 5.0),
 (2883, 'carrot walnut cake', 128473, 1345323600, 5.0),
 (3212, 'pumpkin spice cake in jars', 178427, 1257973200, 5.0),
 (3252, 'crab boil spices', 75218, 1087851600, 5.0),
 (3616, 'oprah s classic mashed potatoes', 2597942, 1399755600, 4.0),
 (3653, 'eggplant  aubergine  and ziti parmesan', 31550, 1177189200, 5.0),
 (3657, 'general tso 

In [26]:
#просто интересно скок строк выполнило условие
query = """SELECT Count(*) From Review
            Inner JOIN Recipe
            ON Recipe.id = Review.id
            WHERE n_ingredients >= 3
            AND review IS NOT NULL 
            AND rating IS NOT NULL 
            AND date IS NOT NULL 
            ORDER BY (Recipe.id) ASC"""
cursor.execute(query)
cursor.fetchall()

[(3429,)]